In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

## Table transactions_without_text

Группировка по client_inn с агрегацией mean по amount

In [4]:
data = pd.read_csv('./Data/correct_data_transaction_data.csv')

In [5]:
data

,date_key,amount,client_inn,partner_inn
0,2018-03-01,41000.00,1279230,1
1,2018-03-01,482300.00,1224261,13022
2,2018-03-01,15.72,1549773,119556
3,2018-03-01,4217742.30,976651,976651
4,2018-03-01,11.77,1413113,1003953
...,...,...,...,...
119833233,2021-03-01,885.61,1739204,656965
119833234,2021-03-01,127165.00,426567,1796124
119833235,2021-03-01,136920.00,1070950,1736097
119833236,2021-03-01,6.00,1387950,119556


In [6]:
#Оставляем только уникальных клиентов (смотрим по client_inn)
data = data.drop_duplicates(subset = ['client_inn','partner_inn'])
data

,date_key,amount,client_inn,partner_inn
0,2018-03-01,41000.00,1279230,1
1,2018-03-01,482300.00,1224261,13022
2,2018-03-01,15.72,1549773,119556
3,2018-03-01,4217742.30,976651,976651
4,2018-03-01,11.77,1413113,1003953
...,...,...,...,...
119833164,2021-03-01,41400.00,1733708,134494
119833186,2021-03-01,128880.00,221040,1368248
119833210,2021-03-01,56000.00,46982,977372
119833215,2021-03-01,6245.35,688085,336836


In [7]:
data['client_inn'].nunique()

64454

In [8]:
#Сохраняем данные только с уникальными клиентами (без дупликатов)
data.to_csv('./Data/no_dupl_transaction_data.csv', index=False)

## Table data_clients

In [9]:
clients = pd.read_csv('/data/home/krazheva/data_clients.csv')
clients = clients.drop(['Клиент_ДатаРег.', 'Клиент_АдресРегистрацииОКАТО', 'Представитель_АдресЖительстваОКАТО', 'Представитель_НачалоДействия', 'Представитель_ОкончаниеДействия'], axis=1)
clients

,inn,Клиент_ДатаНачалаОтн,Клиент_ДатаЗавершОтн,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years
0,1384214,26.03.2021,NaN,МБ,NaN,0
1,286507,25.10.2013,02.07.2019,МКП,NaN,34
2,648912,26.03.2021,NaN,МБ,NaN,0
3,1436001,11.02.2005,NaN,КБ,Д,53
4,918092,26.03.2021,NaN,МКП,NaN,0
...,...,...,...,...,...,...
80089,785208,10.05.2017,03.07.2019,МКП,NaN,37
80090,1206707,20.04.2017,NaN,МКП,Д,38
80091,342755,10.05.2017,NaN,СБ,Д,57
80092,337462,10.05.2017,NaN,МКП,NaN,38


In [10]:
clients['inn'].nunique()

78159

In [11]:
#Оставляем только уникальных клиентов
clients = clients.drop_duplicates(subset = ['inn'])
clients

,inn,Клиент_ДатаНачалаОтн,Клиент_ДатаЗавершОтн,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years
0,1384214,26.03.2021,NaN,МБ,NaN,0
1,286507,25.10.2013,02.07.2019,МКП,NaN,34
2,648912,26.03.2021,NaN,МБ,NaN,0
3,1436001,11.02.2005,NaN,КБ,Д,53
4,918092,26.03.2021,NaN,МКП,NaN,0
...,...,...,...,...,...,...
80089,785208,10.05.2017,03.07.2019,МКП,NaN,37
80090,1206707,20.04.2017,NaN,МКП,Д,38
80091,342755,10.05.2017,NaN,СБ,Д,57
80092,337462,10.05.2017,NaN,МКП,NaN,38


In [12]:
clients.shape

(78159, 6)

In [13]:
#Обработка данных клиентов
clients['Клиент_ДатаЗавершОтн'] = clients['Клиент_ДатаЗавершОтн'].fillna('03.08.2022')
clients['Клиент_ДатаЗавершОтн'] = pd.to_datetime(clients['Клиент_ДатаЗавершОтн'])
clients['Клиент_ДатаНачалаОтн'] = pd.to_datetime(clients['Клиент_ДатаНачалаОтн'])
clients['Длина_Отн_в_днях'] = (clients['Клиент_ДатаЗавершОтн'] - clients['Клиент_ДатаНачалаОтн']).dt.days
clients = clients.drop(['Клиент_ДатаНачалаОтн', 'Клиент_ДатаЗавершОтн'], axis=1)
clients.shape

/tmp/ipykernel_2529380/3182593754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clients['Клиент_ДатаЗавершОтн'] = clients['Клиент_ДатаЗавершОтн'].fillna('03.08.2022')
/tmp/ipykernel_2529380/3182593754.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clients['Клиент_ДатаЗавершОтн'] = pd.to_datetime(clients['Клиент_ДатаЗавершОтн'])
/tmp/ipykernel_2529380/3182593754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

(78159, 5)

In [14]:
clients['Клиент_НаличиеЗПпроекта'] = clients['Клиент_НаличиеЗПпроекта'].fillna('Н')

In [15]:
clients = clients.dropna()
clients.head(2)

,inn,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях
0,1384214,МБ,Н,0,347.0
1,286507,МКП,Н,34,1931.0


In [16]:
clients.shape

(78138, 5)

In [17]:
#Заполнили пропущенные данные 
column_trans = ColumnTransformer(
[('imp_col2', SimpleImputer(missing_values=0, strategy='median'), [3])],
remainder='passthrough')
clients2 = pd.DataFrame(column_trans.fit_transform(clients)[:, [1,2,3,0,4]])
clients2.columns=clients.columns
clients2.index=clients.index
clients2

,inn,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях
0,1384214,МБ,Н,44.0,347.0
1,286507,МКП,Н,34.0,1931.0
2,648912,МБ,Н,44.0,347.0
3,1436001,КБ,Д,53.0,5970.0
4,918092,МКП,Н,44.0,347.0
...,...,...,...,...,...
80089,785208,МКП,Н,37.0,518.0
80090,1206707,МКП,Д,38.0,1783.0
80091,342755,СБ,Д,57.0,1615.0
80092,337462,МКП,Н,38.0,1615.0


In [18]:
#Сделали кодировку категориалльных переменных
one_hot_encoded_data = pd.get_dummies(clients2, columns = ['Клиент_Сегмент'])
one_hot_encoded_data

,inn,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ
0,1384214,Н,44.0,347.0,0,1,0,0
1,286507,Н,34.0,1931.0,0,0,1,0
2,648912,Н,44.0,347.0,0,1,0,0
3,1436001,Д,53.0,5970.0,1,0,0,0
4,918092,Н,44.0,347.0,0,0,1,0
...,...,...,...,...,...,...,...,...
80089,785208,Н,37.0,518.0,0,0,1,0
80090,1206707,Д,38.0,1783.0,0,0,1,0
80091,342755,Д,57.0,1615.0,0,0,0,1
80092,337462,Н,38.0,1615.0,0,0,1,0


In [19]:
clients = pd.get_dummies(one_hot_encoded_data, columns = ['Клиент_НаличиеЗПпроекта'])
clients

,inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н
0,1384214,44.0,347.0,0,1,0,0,0,1
1,286507,34.0,1931.0,0,0,1,0,0,1
2,648912,44.0,347.0,0,1,0,0,0,1
3,1436001,53.0,5970.0,1,0,0,0,1,0
4,918092,44.0,347.0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
80089,785208,37.0,518.0,0,0,1,0,0,1
80090,1206707,38.0,1783.0,0,0,1,0,1,0
80091,342755,57.0,1615.0,0,0,0,1,1,0
80092,337462,38.0,1615.0,0,0,1,0,0,1


In [20]:
#Сохраняем таблицу с данными по клиентам без дубликатов и обработанными данными
clients.to_csv('./Data/GNN_clients_data.csv', index=False)

## Table_data_fraud

In [21]:
fraud = pd.read_csv('/data/home/krazheva/data_fraud.csv')
fraud

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01
...,...,...
6583,86062,2018-12-01
6584,1291546,2019-08-01
6585,684951,2019-10-01
6586,1723386,2019-03-01


In [22]:
fraud_sort = fraud.sort_values(by=['month_fraud'], ascending=False)
fraud_sort

,inn,month_fraud
0,249847,2021-03-01
120,397979,2021-03-01
680,1894908,2021-03-01
184,1409774,2021-03-01
187,1409646,2021-03-01
...,...,...
2767,403952,2018-01-01
1251,884990,2018-01-01
6444,67084,2018-01-01
5633,971420,2018-01-01


In [23]:
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
index1 = fraud_new.iloc[:,0]
index1 = (index1.index.tolist())
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
fraud = fraud.drop(index1)
fraud

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01
...,...,...
6583,86062,2018-12-01
6584,1291546,2019-08-01
6585,684951,2019-10-01
6586,1723386,2019-03-01


In [24]:
fraud = fraud.assign(Fraud_presence = 1)
fraud['inn']=fraud['inn'].astype(int)
fraud

,inn,month_fraud,Fraud_presence
0,249847,2021-03-01,1
1,170435,2020-04-01,1
2,1499321,2019-05-01,1
3,533212,2021-03-01,1
4,1623507,2021-03-01,1
...,...,...,...
6583,86062,2018-12-01,1
6584,1291546,2019-08-01,1
6585,684951,2019-10-01,1
6586,1723386,2019-03-01,1


In [25]:
#Сохраняем обработанные данные без дубликатов
fraud.to_csv('./Data/GNN_data_fraud.csv', index=False)

## Table data reject

In [50]:
reject = pd.read_csv('/data/home/krazheva/data_reject.csv')
reject = reject.assign(Reject_presence = 1)
reject

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
0,826229,ИП,Отказ от закл.дог.,2017-07-01,1
1,183819,ИП,Отказ от закл.дог.,2017-07-01,1
2,1447227,ИП,Отказ от закл.дог.,2017-07-01,1
3,346763,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1
4,1622547,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1
...,...,...,...,...,...
799656,285409,Юридическое лицо,Отказ от закл.дог.,2018-12-01,1
799657,445302,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
799658,421877,ИП,Отказ от закл.дог.,2018-12-01,1
799659,1185066,ИП,Отказ от закл.дог.,2018-12-01,1


In [51]:
reject_new = reject.loc[reject['month_reject'] < '2018-03-01']
index2 = reject_new.iloc[:,0]
index2 = (index2.index.tolist())
reject = reject.drop(index2)
reject

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
115,309291,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
166,1753304,Юридическое лицо,Отказ в проведении операции,2019-01-01,1
197,1238773,ИП,Отказ от закл.дог.,2018-12-01,1
229,1209749,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
248,327939,Юридическое лицо,Отказ от закл.дог.,2019-01-01,1
...,...,...,...,...,...
799656,285409,Юридическое лицо,Отказ от закл.дог.,2018-12-01,1
799657,445302,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
799658,421877,ИП,Отказ от закл.дог.,2018-12-01,1
799659,1185066,ИП,Отказ от закл.дог.,2018-12-01,1


In [52]:
#Сохраняем обработанные данные без дубликатов
reject.to_csv('./Data/GNN_data_reject.csv', index=False)

## Объединение всех данных в одну таблицу

In [29]:
'''
В операции merge, default параметр how=inner, то есть происходит соединение только
совпадающих строк, а несовпадающие строки удаляются из таблицы. Поэтому, после операции merge
получается датасет, в котором ко всем строкам таблицы транзакций подходят все инн из таблицы клиентов.
То есть у нас нет клиентов не банка (не из таблицы clients), так как когда мы мерджили мы взяли 
все строки, которые присутствуют в clients таблице.
'''

'\nВ операции merge, default параметр how=inner, то есть происходит соединение только\nсовпадающих строк, а несовпадающие строки удаляются из таблицы. Поэтому, после операции merge\nполучается датасет, в котором ко всем строкам таблицы транзакций подходят все инн из таблицы клиентов.\nТо есть у нас нет клиентов не банка (не из таблицы clients), так как когда мы мерджили мы взяли \nвсе строки, которые присутствуют в clients таблице.\n'

In [31]:
data.shape

(6930168, 4)

In [32]:
clients.shape

(78138, 9)

In [ ]:
step1 = pd.merge(left=data, right=clients, left_on='client_inn', right_on='inn')

In [ ]:
step1.shape

(6833282, 13)

In [ ]:
step1

,date_key,amount,client_inn,partner_inn,inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н
0,2018-03-01,41000.00,1279230,1,1279230,54.0,6251.0,1,0,0,0,0,1
1,2018-03-01,618280.00,1279230,1003953,1279230,54.0,6251.0,1,0,0,0,0,1
2,2018-03-01,2698540.00,1279230,682959,1279230,54.0,6251.0,1,0,0,0,0,1
3,2018-03-01,3000.00,1279230,1852820,1279230,54.0,6251.0,1,0,0,0,0,1
4,2018-03-01,1000.00,1279230,119556,1279230,54.0,6251.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833277,2021-03-01,1000.00,673145,1003953,673145,47.0,127.0,0,0,1,0,0,1
6833278,2021-03-01,290.00,764479,119556,764479,46.0,375.0,0,0,1,0,0,1
6833279,2021-03-01,2000.00,398315,119556,398315,34.0,126.0,0,1,0,0,0,1
6833280,2021-03-01,10855.68,1095817,1095817,1095817,42.0,187.0,0,0,1,0,1,0


In [38]:
(step1['inn'].isna()).unique()

array([False])

In [39]:
step1['inn'].unique().shape

(63864,)

In [40]:
step1 = step1.drop(['inn'], axis=1)
step1

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6833277,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1
6833278,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1
6833279,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1
6833280,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0


In [41]:
'''
В операции merge_outer defaul в качестве параметра how=outer. Поэтому, при объединении таблиц может 
получиться размерность итоговой таблицы б ольше размерности обеих объединяющихся таблиц. 
'''

#Это нужно так как если нет информации, то мы заполняем ее нулями, так как это значит, что мошенничества нет
#Если в таблице Fraud будут новые inn, то все остальные колонки мы заполним нулями, и потом удалим эти строки на последующих шагах

'\nВ операции merge_outer defaul в качестве параметра how=outer. Поэтому, при объединении таблиц может \nполучиться размерность итоговой таблицы б ольше размерности обеих объединяющихся таблиц. \n'

In [42]:
fraud.rename(columns = {'inn':'client_inn'}, inplace = True )
fraud.shape

(6400, 3)

In [43]:
fraud = fraud.drop_duplicates(subset=['client_inn']) 
fraud.shape

(6299, 3)

In [44]:
step2 = step1.merge(fraud, how = 'left', on='client_inn')#left=step1, right=fraud, left_on='client_inn', right_on='inn')
step2 = step2.drop(['month_fraud'], axis=1)
step2['Fraud_presence'] = step2['Fraud_presence'].fillna(0)
step2

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833277,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0
6833278,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0
6833279,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0
6833280,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0


In [45]:
step2.dropna()
step2

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833277,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0
6833278,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0
6833279,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0
6833280,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0


In [46]:
# Должен быть пустой датафрейм
duplicateRowsstep2 = step2[step2.duplicated ()]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(duplicateRowsstep2)

Empty DataFrame
Columns: [date_key, amount, client_inn, partner_inn, years, Длина_Отн_в_днях, Клиент_Сегмент_КБ, Клиент_Сегмент_МБ, Клиент_Сегмент_МКП, Клиент_Сегмент_СБ, Клиент_НаличиеЗПпроекта_Д, Клиент_НаличиеЗПпроекта_Н, Fraud_presence]
Index: []


In [47]:
step2['client_inn'].unique().shape

(63864,)

In [53]:
reject.rename(columns = {'inn':'client_inn'}, inplace = True )
reject.shape

(546684, 5)

In [54]:
step3 = step2.merge(reject, how = 'left', on='client_inn')
step3 = step3.drop(['РеестрОтказов_ТипУчастника', 'РеестрОтказов_КодОтказаСтрокой', 'month_reject'], axis=1)
step3['Reject_presence'] = step3['Reject_presence'].fillna(0)

In [55]:
step3

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,0.0
7015933,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0,1.0
7015934,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,0.0
7015935,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,0.0


In [68]:
#step3 = step3.drop_duplicates(subset=['client_inn'], keep='last')
#step3

In [56]:
final_data = step3

In [57]:
final_data = step3.dropna(thresh=8)
final_data

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,0.0
7015933,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0,1.0
7015934,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,0.0
7015935,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,0.0


In [72]:
'''
В результате мы все равно получили число строк такое же, как при объединении двух таблиц
data и clients методом inner. То есть у нас все строки, которые сейчас присутствуют имеются в таблице 
clients. Поэтому все пользователи являются клиентами банка.
'''

'\nВ результате мы все равно получили число строк такое же, как при объединении двух таблиц\ndata и clients методом inner. То есть у нас все строки, которые сейчас присутствуют имеются в таблице \nclients. Поэтому все пользователи являются клиентами банка.\n'

In [58]:
#Сохраняем таблицу с данными по клиентам без дубликатов и обработанными данными
final_data.to_csv('./Data/GNN_final_data.csv', index=False)